### Set THRESHOLD_RANK, NEIGHBORS_ONLY, PPR_THRESHOLD_ONLY in communities.py accordingly.

In [1]:
import json
import random
import os
import pickle
import time
import shutil
import sys
import uuid
from collections import defaultdict
from datetime import timedelta
from glob import glob

import leidenalg as la
import igraph as ig
import numpy as np
import pandas as pd
import seaborn as sns
import psutil

import settings as s
from common import get_processes

%load_ext autoreload
%autoreload 2

In [2]:
sns.set_theme(style="white", context="talk")

In [3]:
if (
    sys.version_info.major,
    sys.version_info.minor,
    sys.version_info.micro,
) != (3, 11, 8):
    raise EnvironmentError(
        "Only runs efficiently on Python 3.11.8 (Tested on: Conda 24.1.2 | Apple M3 Pro)"
    )

In [4]:
start_script = time.time()

In [5]:
data = pd.read_csv("./open-data/Libra_bank_3months_graph/data.csv")
rename = {
    "id_source": "source",
    "id_destination": "target",
    "cum_amount": "amount",
    "nr_transactions": "transactions_count",
    "nr_alerts": "alerts_count",
    "nr_reports": "reports_count",
}
data = data.rename(columns=rename)
data.loc[:, "source_"] = (
    data.loc[:, "source"].astype(str).apply(lambda x: f"nid-{int(x)}")
)
data.loc[:, "target_"] = (
    data.loc[:, "target"].astype(str).apply(lambda x: f"nid-{int(x)}")
)
del data["source"]
del data["target"]
data = data.rename(
    columns={
        "source_": "source",
        "target_": "target",
    }
).loc[:, ["source", "target", "amount", "transactions_count", "alerts_count", "reports_count"]]

In [6]:
nodes = pd.DataFrame(set(data["source"].tolist() + data["target"].tolist()), columns=["id"])

w_alerts = int(data["alerts_count"].sum() * 2)
w_reports = int(data["reports_count"].sum() * 2)

w_alerts_source = data[data["alerts_count"] > 0].groupby("source").agg({"alerts_count": "sum"}).to_dict()["alerts_count"]
w_alerts_target = data[data["alerts_count"] > 0].groupby("target").agg({"alerts_count": "sum"}).to_dict()["alerts_count"]

w_reports_source = data[data["reports_count"] > 0].groupby("source").agg({"reports_count": "sum"}).to_dict()["reports_count"]
w_reports_target = data[data["reports_count"] > 0].groupby("target").agg({"reports_count": "sum"}).to_dict()["reports_count"]

nodes.loc[:, "alert_weight"] = nodes.loc[:, "id"].apply(
    lambda x: (w_alerts_source.get(x, 0) + w_alerts_target.get(x, 0)) / w_alerts
)
nodes.loc[:, "report_weight"] = nodes.loc[:, "id"].apply(
    lambda x: (w_reports_source.get(x, 0) + w_reports_target.get(x, 0)) / w_reports
)

In [7]:
nodes.sort_values("alert_weight")

,id,alert_weight,report_weight
0,nid-84882,0.000000,0.0
256655,nid-192899,0.000000,0.0
256654,nid-118873,0.000000,0.0
256653,nid-231162,0.000000,0.0
256652,nid-342556,0.000000,0.0
...,...,...,...
280692,nid-28317,0.007737,0.0
345660,nid-119992,0.008704,0.0
146626,nid-356200,0.009671,0.0
324001,nid-364511,0.009671,0.0


In [ ]:
shortest_paths = sub_g.get_shortest_paths(
                    node, to=neighborhood, weights="weight", mode="all"
                )

In [7]:
def aggregate_edges(data_input):
    source_totals = (
        data_input.groupby("source")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )
    target_totals = (
        data_input.groupby("target")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )
    data_aggregated = data_input.copy(deep=True)
    data_aggregated.loc[:, "total_sent_by_source"] = data_aggregated.loc[
        :, "source"
    ].apply(lambda x: source_totals[x])
    data_aggregated.loc[:, "total_received_by_target"] = data_aggregated.loc[
        :, "target"
    ].apply(lambda x: target_totals[x])
    data_aggregated.loc[:, "weight"] = data_aggregated.apply(
        lambda x: (
            (x["amount"] / x["total_sent_by_source"])
            + (x["amount"] / x["total_received_by_target"])
        ),
        axis=1,
    )
    return data_aggregated.loc[:, ["source", "target", "weight"]]

In [8]:
%%time

edges = aggregate_edges(data)
graph = ig.Graph.DataFrame(edges, use_vids=False, directed=True)
nodes = [x["name"] for x in graph.vs()]
random.shuffle(nodes)

CPU times: user 2.88 s, sys: 51.9 ms, total: 2.93 s
Wall time: 2.94 s


In [9]:
raise

RuntimeError: No active exception to reraise

In [ ]:
nodes_mapping = {x.index: x["name"] for x in graph.vs()}

In [ ]:
%%time

NUMBER_OF_PROCESSES = 10

shutil.rmtree("staging", ignore_errors=True)
os.mkdir("staging")
chunks = np.array_split(nodes, NUMBER_OF_PROCESSES)

filename = "graph.pickle"
with open(filename, "wb") as f:
    pickle.dump(graph, f, protocol=pickle.HIGHEST_PROTOCOL)

filename = "nodes.pickle"
with open(filename, "wb") as f:
    pickle.dump(chunks, f, protocol=pickle.HIGHEST_PROTOCOL)

process_ids = set()
process_name = "communities_libra.py"
for chunk_number in range(NUMBER_OF_PROCESSES):
    process_id = str(uuid.uuid4())
    process_ids = process_ids.union({process_id})
    os.system(f"{sys.executable} {process_name} {chunk_number} {process_id} &")

while get_processes(process_ids):
    time.sleep(5)

In [ ]:
%%time

for proc in get_processes(process_ids):
    try:
        proc.kill()
    except psutil.NoSuchProcess:
        pass

communities = []
for filename in glob("./staging/*.pickle"):
    with open(filename, "rb") as f:
        communities += pickle.load(f)

original_size = len(communities)

filename = "communities.pickle"
with open(filename, "wb") as f:
    pickle.dump(communities, f)

communities_unique_rev = {}
for comm_id, comm in communities:
    communities_unique_rev[tuple(sorted(comm))] = comm_id

communities_unique = {
    f"{v}-{i}": k for i, (k, v) in enumerate(communities_unique_rev.items())
}
communities = list({x for x in communities_unique.values()})
new_size = len(communities)

print(original_size, new_size, round(new_size / original_size, 2))
print()

In [ ]:
sizes = [len(x) for x in communities]
round(np.mean(sizes)), round(np.max(sizes)), sum(sizes)

In [ ]:
np.percentile(sizes, 25), np.percentile(sizes, 50), np.percentile(sizes, 75)

In [ ]:
## sns.set_theme(rc={"figure.figsize":(12.7, 7.27)})
sns.histplot(data=pd.DataFrame(sizes, columns=["Size"]), x="Size", kde=True)

In [ ]:
delta = round(time.time() - start_script)
print(f"Script executed in {timedelta(seconds=delta)}")